In [ ]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison
import sys
from boruta import BorutaPy

In [ ]:
inpath = sys.argv[1]
file_name=inpath.split('/')[9].split('.')[0]
m = pd.read_csv(inpath, delimiter="\t", index_col=0, header=None)

In [ ]:
m = m.T
X = m.loc[:, m.columns != 'sample']  # data
X = X.T  # index=cg
y = m.loc[:, m.columns == 'sample'].values.ravel()  # label
df = pd.DataFrame()
f_g1=[]
f_g2=[]
f_g3=[]
f_g4=[]
f_g5=[]
f_g6=[]
f_g7=[]
f_g8=[]
f_g9=[]
f_g10=[]
f_g11=[]

In [ ]:
for index,row in X.iterrows():  ##ANOVA, Tukey-kramer
    temp = pd.DataFrame()
    buff = pd.to_numeric(list(row))
    CpG = index  
    temp['beta'] = buff
    temp['Type'] = y  # label
    model = ols('beta~Type', data=temp).fit()
    anovat = anova_lm(model)
    anovat_p = anovat['PR(>F)'][0]
    if anovat_p<0.05:
        mc = MultiComparison(temp['beta'], temp['Type'])
        mc_results = mc.tukeyhsd(alpha=0.01)  # unequal sample size data, tukey_hsd uses Tukey-Kramer test
        result = pd.DataFrame(mc_results._results_table.data[1:len(mc_results._results_table.data)],
                              columns=mc_results._results_table.data[0])
        result['CpG'] = CpG
        n=result[result['reject']==True]

        df = pd.concat([df, n])

        condition = abs(n['meandiff'])>= 0.2
        n=n[condition]
        list_a = n['group1'].tolist()
        list_b = n['group2'].tolist()
        list_a.extend(list_b)
        if list_a.count('group-1')>=11:
            f_g1.append(CpG)
        if list_a.count('group-2')>=11:
            f_g2.append(CpG)
        if list_a.count('group-3')>=11:
            f_g3.append(CpG)
        if list_a.count('group-4')>=11:
            f_g4.append(CpG)
        if list_a.count('group-5')>=11:
            f_g5.append(CpG)
        if list_a.count('group-6')>=11:
            f_g6.append(CpG)
        if list_a.count('group-7')>=11:
            f_g7.append(CpG)
        if list_a.count('group-8')>=11:
            f_g8.append(CpG)
        if list_a.count('group-9')>=11:
            f_g9.append(CpG)
        if list_a.count('group-10')>=11:
            f_g10.append(CpG)
        if list_a.count('group-11')>=11:
            f_g11.append(CpG)
        """if list_a.count('group-12')>=11:
            f_g12.append(CpG)"""
    else:
        continue

In [ ]:
f=open('/group1-F/'+file_name+'_1.txt','w')##download the markers after ANOVA, Tukey-Kramer test
#f=open('C:\\Users\\Think\\Desktop\\1.txt','w')
f.write(str(f_g1))
f.close()
f=open('group2-F/'+file_name+'_2.txt','w')
#f=open('C:\\Users\\Think\\Desktop\\2.txt','w')
f.write(str(f_g2))
f.close()
f=open('group3-F/'+file_name+'_3.txt','w')
f.write(str(f_g3))
f.close()
f=open('group4-F/'+file_name+'_4.txt','w')
f.write(str(f_g4))
f.close()
f=open('group5-F/'+file_name+'_5.txt','w')
f.write(str(f_g5))
f.close()
f=open('group6-F/'+file_name+'_6.txt','w')
f.write(str(f_g6))
f.close()
f=open('group7-F/'+file_name+'_7.txt','w')
f.write(str(f_g7))
f.close()
f=open('group8-F/'+file_name+'_8.txt','w')
f.write(str(f_g8))
f.close()
f=open('group9-F/'+file_name+'_9.txt','w')
f.write(str(f_g9))
f.close()
f=open('group10-F/'+file_name+'_10.txt','w')
f.write(str(f_g10))
f.close()
f=open('group11-F/'+file_name+'_11.txt','w')
f.write(str(f_g11))
f.close()
f=open('group12-F/'+file_name+'_12.txt','w')
f.write(str(f_g12))
f.close()
df.to_csv('tukey_val/'+file_name+'Tukey_all.csv', sep='\t', na_rep='NA')


Next, the sample after the ANOVA test, Tukey-kramer, is read for Boruta feature selection

In [ ]:
m = pd.read_csv(sample, delimiter="\t", index_col=0, header=None).T
f = open('group_12.txt', 'r')
fl = eval(f.read())
feature_list=['sample']
feature_list.extend(fl)
m=m.T.loc[feature_list,:]
X = m.loc[:, m.columns != '0']
y = m.loc[:, m.columns == '0'].values.ravel()

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=12,n_jobs=-1,class_weight='balanced') #Boruta
feat_selector = BorutaPy(rfc, n_estimators='auto', verbose=2)
feat_selector.fit(np.array(X), np.array(y))

In [ ]:
a = m.columns[1:]#download marker after boruta
print(a[feat_selector.support_].tolist())
print(len(a[feat_selector.support_]))
f = open("F:\\classifiation\\group11_tubo.txt", 'w')
f.write(str(a[feat_selector.support_].tolist()))
f.close()